In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pprint as pp

In [ ]:
review = pd.read_csv("../review.csv")

In [32]:
df = pd.read_csv("goodreads.csv")

In [ ]:
df['book_title']

- There are 1246 unique books
- Books with the most number of reviews: ['Inferno', 'Artemis Fowl', 'The Queen of the Damned',
       'Eat, Pray, Love: One Woman's Search for Everything Across Italy, India and Indonesia',
       'The 5th Wave']

In [27]:
pp.pprint(review['book_title'].value_counts()[:5].index)
review['book_title'].unique().size

Index(['Inferno', 'Artemis Fowl', 'The Queen of the Damned',
       'Eat, Pray, Love: One Woman's Search for Everything Across Italy, India and Indonesia',
       'The 5th Wave'],
      dtype='object')


1246